In [1]:
#@title Installing packages, doing imports & downloading weights

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install ftfy==5.8
!pip install transformers

import os
import random

import numpy as np
import PIL
import torch

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip

!git clone https://github.com/FreddeFrallan/Multilingual-CLIP
%cd Multilingual-CLIP
!bash get-weights.sh

CUDA version: 11.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.3MB/s eta 0:04:12tcmalloc: large alloc 1147494400 bytes == 0x55a6ac846000 @  0x7fc57e92c615 0x55a6736f406c 0x55a6737d3eba 0x55a6736f6e8d 0x55a6737e899d 0x55a67376afe9 0x55a673765b0e 0x55a6736f877a 0x55a67376ae50 0x55a673765b0e 0x55a6736f877a 0x55a67376786a 0x55a6737e97c6 0x55a673766ee2 0x55a6737e97c6 0x55a673766ee2 0x55a6737e97c6 0x55a673766ee2 0x55a6737e97c6 0x55a67386b431 0x55a6737cc049 0x55a673736c84 0x55a6736f78e9 0x55a67376bade 0x55a6736f869a 0x55a673766a45 0x55a673765e0d 0x55a6736f877a 0x55a673766a45 0x55a6736f869a 0x55a673766a45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:28tcmalloc: large alloc 1434370048 bytes == 0x55a6f0e9c000 @  0x7fc57e92c615 0x55a6736f406c 0x55a6737d3eba 0x55a6736f6e8d 0x55a6737e899d 0x55a67376afe9 0x55a673765b0e 0x55a6736f877a 0x55a67376ae50 0x55a673765b0e 0x55a6736f877a 0x55a67376786a 0x55a6737e97

In [2]:
from src.multilingual_clip import MultilingualClip

model_path = 'M-CLIP/Swedish-500k'
tok_path = 'M-CLIP/Swedish-500k'
head_weight_path = 'data/weights/Swedish-500k Linear Weights.pkl'

sweclip_args = {'model_name': model_path,
                'tokenizer_name': tok_path,
                'head_path': head_weight_path}

sweclip = MultilingualClip(**sweclip_args)